In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from data_utils import CustomTransform
import numpy as np
import wandb
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import precision_recall_curve, average_precision_score, PrecisionRecallDisplay
import matplotlib.pyplot as plt
import tqdm
import faiss
import pickle

In [2]:
# Define a function to extract features from an image
def extract_features(loader, model, device):
    with torch.no_grad():    
        features, labels = [], []
        for img, label in tqdm.tqdm(loader, total=len(loader), desc='Extracting features'):
            features.append(model(img.to(device)).cpu().detach().numpy())
            labels.append(label)
    return np.concatenate(features).astype('float32'), np.concatenate(labels)


# k-NN Classifier for Image Retrieval
class ImageRetrievalSystem:
    def __init__(self, model, train_dataloader, test_dataloader, config):
        self.model = model
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.device = config['device']

        self.dim = 2048
        self.classifier_type = config['classifier']
        self.n_neighbors = config['n_neighbors']

        if self.classifier_type == 'knn':
            self.classifier = NearestNeighbors(n_neighbors=config['n_neighbors'], metric=config['metric'])
        else:
            self.classifier = faiss.IndexIVFFlat(faiss.IndexFlatL2(self.dim), self.dim, config['voronoi_cells'])
            self.classifier.nprobe = config['lookup_cells']

    def fit(self, precomputed=False):

        if not precomputed:
            features, self.train_labels = extract_features(self.train_dataloader, self.model, self.device)

            with open('train_features.pkl', 'wb') as f:
                pickle.dump((features, self.train_labels), f)

        else:
            with open('train_features.pkl', 'rb') as f:
                features, self.train_labels = pickle.load(f)
        
        print('Fitting the classifier...')
        if self.classifier_type == 'knn':
            self.classifier.fit(features, self.train_labels)
        else:
            self.classifier.train(features)
            self.classifier.add(features)

    def retrieve(self, precomputed=False):

        if not precomputed:
            features, labels = extract_features(self.test_dataloader, self.model, self.device)

            with open('test_features.pkl', 'wb') as f:
                pickle.dump((features, labels), f)

        else:
            with open('test_features.pkl', 'rb') as f:
                features, labels = pickle.load(f)
        
        print('Retrieving images...')
        if self.classifier_type == 'knn':
            _, predictions = self.classifier.kneighbors(features, return_distance=True)
        else:
            _, predictions = self.classifier.search(features, self.n_neighbors)
        
        return predictions, labels

In [3]:
def evaluate(predictions, labels, config):

    # Prec@1
    prec_at_1 = np.mean([1 if predictions[i, 0] == labels[i] else 0 for i in range(predictions.shape[0])])

    # Prec@5
    prec_at_5 = np.mean([np.sum([1 if predictions[i, j] == labels[i] else 0 for j in range(config['n_neighbors'])]) / config['n_neighbors'] for i in range(predictions.shape[0])])

    # Initialize list to store average precision for each query
    average_precisions = []

    # Compute binary relevance arrays and calculate average precision for each query
    for i in range(predictions.shape[0]):
        # Convert true label into binary format for each prediction
        binary_relevance = np.array([1 if label == labels[i] else 0 for label in predictions[i]])
        
        # Ensure there is at least one positive class to avoid division by zero in AP score calculation
        if np.sum(binary_relevance) > 0:
            # Compute the average precision for the current query
            ap_score = average_precision_score(binary_relevance, np.ones_like(binary_relevance))
            average_precisions.append(ap_score)

    # Compute mean Average Precision (mAP) by averaging all the AP scores
    mean_ap = np.mean(average_precisions)

    return prec_at_1, prec_at_5, mean_ap

In [4]:
def sweep():
    with wandb.init() as run:
        # Get hyperparameters
        config = run.config
        torch.manual_seed(123) # seed for reproductibility

        transform_train = CustomTransform(config, mode='train')

        train_dataset = datasets.ImageFolder(root=config['TRAINING_DATASET_DIR'], transform=transform_train)

        total_length = len(train_dataset)
        train_size = int(0.8 * total_length)  # e.g., 80% for training
        valid_size = total_length - train_size  # remaining 20% for validation

        train_dataset, validation_dataset = random_split(train_dataset, [train_size, valid_size])

        dataloader_train = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        dataloader_validation = DataLoader(validation_dataset, batch_size=config['batch_size'], shuffle=True)

        model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights='ResNet50_Weights.DEFAULT').eval()
        model.fc = nn.Identity()
        model = model.to(config['device'])
        
        pipeline = ImageRetrievalSystem(model, dataloader_train, dataloader_validation, config)
        pipeline.fit(precomputed=True)
        predictions, labels = pipeline.retrieve(precomputed=True)

        predictions = pipeline.train_labels[predictions]

        prec_at_1, prec_at_5, mean_ap = evaluate(predictions, labels, config)
        
        wandb.log({"prec_at_1": prec_at_1, "prec_at5": prec_at_5, "mean_ap": mean_ap})


In [5]:
sweep_configuration = {
    "method": "grid",
    "name": "sweep1",
    "metric": {"goal": "maximize", "name": "mean_ap"},
    "parameters": {
        'n_neighbors': {'values': [3, 5, 10, 15, 20]},
        'metric': {'values': ['euclidean', 'manhattan', 'minkowski']},
        'IMG_WIDTH': {'value': 256},
        'IMG_HEIGHT': {'value': 256},
        'TRAINING_DATASET_DIR': {'value': '../Week 1/data/MIT_split/train'},
        'TEST_DATASET_DIR': {'value': '../Week 1/data/MIT_split/test'},
        'batch_size': {'value': 32},
        'classifier': {'value': 'knn'},
        'voronoi_cells': {'value': 64},
        'lookup_cells': {'value': 8},
        'device': {'value': "cpu"}
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="week3_task1.1", entity="c5-g8")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 0lwk44ks
Sweep URL: https://wandb.ai/c5-g8/week3_task1.1/sweeps/0lwk44ks


In [6]:
sweep_configuration = {
    "method": "grid",
    "name": "sweep1",
    "metric": {"goal": "maximize", "name": "mean_ap"},
    "parameters": {
        'n_neighbors': {'value': 3},
        'metric': {'value': 'euclidean'},
        'IMG_WIDTH': {'value': 256},
        'IMG_HEIGHT': {'value': 256},
        'TRAINING_DATASET_DIR': {'value': '../Week 1/data/MIT_split/train'},
        'TEST_DATASET_DIR': {'value': '../Week 1/data/MIT_split/test'},
        'batch_size': {'value': 32},
        'classifier': {'value': 'fais'},
        'voronoi_cells': {'values': [32, 64, 128, 256]},
        'lookup_cells': {'values': [4, 8, 16, 32]},
        'device': {'value': "cpu"}
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="week3_task1.1", entity="c5-g8")

Create sweep with ID: eayw9nmw
Sweep URL: https://wandb.ai/c5-g8/week3_task1.1/sweeps/eayw9nmw


In [7]:
wandb.agent('c5-g8/week3_task1.1/eayw9nmw', function=sweep, count=16)

wandb: Agent Starting Run: xfix7v1h with config:
wandb: 	IMG_HEIGHT: 256
wandb: 	IMG_WIDTH: 256
wandb: 	TEST_DATASET_DIR: ../Week 1/data/MIT_split/test
wandb: 	TRAINING_DATASET_DIR: ../Week 1/data/MIT_split/train
wandb: 	batch_size: 32
wandb: 	classifier: fais
wandb: 	device: cpu
wandb: 	lookup_cells: 4
wandb: 	metric: euclidean
wandb: 	n_neighbors: 3
wandb: 	voronoi_cells: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: caguilera1401 (c5-g8). Use `wandb login --relogin` to force relogin


Using cache found in C:\Users\cagui/.cache\torch\hub\pytorch_vision_v0.10.0


Fitting the classifier...
Retrieving images...
